# Imports

In [330]:
import os
import sys
import time
import logging
import warnings
from datetime import datetime, timedelta
from collections import defaultdict
from io import StringIO

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

plt.rcParams.update({
    "figure.figsize": (16, 9),
    "font.size": 12,
    "figure.dpi": 300,
    "savefig.dpi": 300
})

#### Logging

In [331]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S",
)
log = logging.getLogger("ElectoralCycles")

### Global Configs

In [332]:
ARQUIVO_ENTRADA = "resultados_analise_b3_com_tickers.xlsx"
SHEET_NAME = "LISTA FINAL (Cont+IPOs-Canc)"
OUTPUT_DIR = "./output_v3"

# ---- Datas eleitorais (1º e 2º turnos) -----------------------------------
DATAS_PRIMEIRO_TURNO = {
    2002: pd.Timestamp("2002-10-06"),
    2006: pd.Timestamp("2006-10-01"),
    2010: pd.Timestamp("2010-10-03"),
    2014: pd.Timestamp("2014-10-05"),
    2018: pd.Timestamp("2018-10-07"),
    2022: pd.Timestamp("2022-10-02"),
}
DATAS_SEGUNDO_TURNO = {
    2002: pd.Timestamp("2002-10-27"),
    2006: pd.Timestamp("2006-10-29"),
    2010: pd.Timestamp("2010-10-31"),
    2014: pd.Timestamp("2014-10-26"),
    2018: pd.Timestamp("2018-10-28"),
    2022: pd.Timestamp("2022-10-30"),
}
ANOS_ELEITORAIS = sorted(DATAS_PRIMEIRO_TURNO.keys())

### Setores

In [333]:
# ---- Setores B3 ----------------------------------------------------------
SETORES_B3 = [
    "Bens Industriais", "Comunicações", "Construção e Transporte",
    "Consumo Cíclico", "Consumo Não Cíclico", "Financeiro",
    "Materiais Básicos", "Outros", "Petróleo, Gás e Biocombustíveis",
    "Saúde", "Utilidade Pública",]
# Tickers importantes de Petróleo que vamos forçar em todos os anos
TICKERS_PETROLEO_FORCADOS = {
    2002: ["PETR4.SA", "PETR3.SA"],
    2006: ["PETR4.SA", "PETR3.SA"],
    2010: ["PETR4.SA", "PETR3.SA"],
    2014: ["PETR4.SA", "PETR3.SA", "PRIO3.SA"],
    2018: ["PETR4.SA", "PETR3.SA", "PRIO3.SA", "VBBR3.SA"],
    2022: ["PETR4.SA", "PETR3.SA", "PRIO3.SA", "VBBR3.SA", "RRRP3.SA", "CSAN3.SA"]
}

Δ = CAR_estatais − CAR_privadas

Se Δ for significativo, isso sugere que o efeito vem especificamente do risco político/regulatório e não de um choque geral que afeta todo mundo igual.



### PARÂMETROS DE JANELAS, FILTRAGEM E ROBUSTEZ


In [334]:
# ---- Janela de estimação (MacKinlay, 1997) --------------------------------
ESTIMACAO_INICIO_DU = -252    # dias úteis antes do 1º turno
ESTIMACAO_FIM_DU = -30        # dias úteis antes do 1º turno

# ---- Janelas de evento (dias úteis relativos ao 1º turno) ----------------
JANELAS_EVENTO_1T = {
    "antecipacao_45":  (-45, -1),     # Efeito propaganda / pricing-in
    "antecipacao_60":  (-60, -1),     # Robustez: janela mais larga
    "reacao_curta_1t": (-5, +5),      # Reação imediata ao 1º turno
    "reacao_media_1t": (-10, +10),    # Robustez
    "reacao_ampla_1t": (-20, +20),    # Robustez: persistência
}

# ---- Janelas relativas ao 2º turno --------------------------------------
JANELAS_EVENTO_2T = {
    "reacao_curta_2t": (-5, +5),      # Reação imediata ao 2º turno
    "reacao_media_2t": (-10, +10),    # Robustez: digestão do resultado
}


Janela "entre turnos" De [-5 d.u. antes do 1ºT] até [-1 d.u. antes do 2ºT]

Período de máxima incerteza: mercado sabe os finalistas mas não o resultado.

Tamanho variável (~15-18 d.u. por ano). O N de dias é registrado no output.


In [335]:
JANELA_ENTRE_TURNOS = True    # flag para ativar

# ---- Janelas semestrais (ciclo interno) ----------------------------------
JANELAS_CICLO = True          # 1º sem (expectativa) vs 2º sem (disputa)

# ---- Janela estendida ----------------------------------------------------
JANELA_ESTENDIDA = True       # últimos 6 meses do ano eleitoral

JANELAS_ESPELHO = True


ANOS-ESPELHO:

   Ano eleitoral → Espelho (T-1) → Contexto do espelho

   2002         → 2001           → Pós-crise Argentina, 11 de setembro

   2006         → 2005           → Mensalão (crise política, não eleitoral)

   2010         → 2009           → Recuperação pós-crise 2008

   2014         → 2013           → Jornadas de Junho (protestos em massa)

   2018         → 2017           → Joesley Day (mai/17), recuperação no 2º sem
   
   2022         → 2021           → Pós-COVID, boom de IPOs, Selic subindo

 Nenhum ano-espelho é perfeitamente "normal". Os resultados devem ser
interpretados com essa ressalva. Reportar N e contexto em tabela separada.

## Parâmetros

In [336]:
MIN_PREGOES_PCT = 0.40        # Mínimo 80% de pregões no ano para inclusão
MIN_EMPRESAS_SETOR = 1        # Corte: mínimo N empresas por setor/ano
ANOS_CRISE = [2008, 2014, 2020]    
N_PLACEBO_EVENTS = 1000          



## Selic

In [337]:
SELIC_ANUAL = {
    2002: 0.1911, 2006: 0.1513, 2010: 0.0975,
    2014: 0.1115, 2018: 0.0640, 2022: 0.1275,
}
IPCA_ANUAL = {
    2002: 0.1253, 2006: 0.0314, 2010: 0.0591,
    2014: 0.0641, 2018: 0.0375, 2022: 0.0562,
}

# Selic diária (proxy CDI) para cálculo de Sharpe
SELIC_DIARIA = {k: (1 + v) ** (1/252) - 1 for k, v in SELIC_ANUAL.items()}


 NEFIN (Núcleo de Economia Financeira da USP) calcula diariamente os
fatores de risco Fama-French adaptados para o Brasil:

- Mkt-Rf : prêmio de risco de mercado (retorno mercado − CDI)

- SMB    : prêmio de tamanho (small caps − large caps)

- HML    : prêmio de valor (alto B/M − baixo B/M)

- WML    : prêmio de momentum (vencedoras − perdedoras)

- IML    : prêmio de iliquidez (ilíquidas − líquidas)   - Rf     : taxa livre de risco diária (CDI)


samos para rodar um modelo alternativo de robustez (Fama-French 3):

R_i − Rf = α + β·MktRf + s·SMB + h·HML + ε

Se o CAR permanece significativo após controlar por tamanho e valor, o resultado é mais robusto contra a crítica de que o efeito capturado

seria apenas exposição a small caps ou empresas de valor.


Se o download falhar (site fora, sem internet), o script continua normalmente apenas com o Modelo de Mercado (CAPM).

In [338]:
#Corrigir alguns apenas

TICKER_MAPPING = {
    "VVAR3": "BHIA3", "BTOW3": "AMER3", "LAME4": "LAME3", "PCAR4": "PCAR3",
    "KROT3": "COGN3", "ESTC3": "YDUQ3", "RAIL3": "RUMO3",
    "BVMF3": "B3SA3", "CTIP3": "B3SA3",
    "BRIN3": "BRML3", "BRML3": "ALOS3", "SMLE3": "SMFT3",
    "LINX3": "STNE3", "VIVT4": "VIVT3", "TIMP3": "TIMS3",
    "QGEP3": "BRAV3", "GNDI3": "HAPV3", "FIBR3": "SUZB3",
}

## ETAPA 1 — Ingestão de Dados

In [339]:
def carregar_lista_empresas(caminho: str) -> pd.DataFrame:
    """
    Carrega lista de empresas, aplica mapeamento De-Para,
    e lê coluna ESTATAL direto da planilha.
    """
    log.info("=" * 70)
    log.info("ETAPA 1: INGESTÃO DE DADOS")
    log.info("=" * 70)

    df = pd.read_excel(caminho, sheet_name="LISTA FINAL (Cont+IPOs-Canc)")
    df = df.dropna(subset=["TICKER", "SETOR_B3"])

    # Parse de datas
    df["DT_REG"] = pd.to_datetime(df["DT_REG"], errors="coerce")

    # Coluna ESTATAL: converte "Sim"/"Não" para booleano
    df["ESTATAL"] = df["ESTATAL"].str.strip().str.upper().eq("SIM")

    # Ticker original → mapeado → yfinance
    df["TICKER_ORIGINAL"] = df["TICKER"].str.strip()
    df["TICKER_MAPEADO"] = df["TICKER_ORIGINAL"].map(TICKER_MAPPING).fillna(
        df["TICKER_ORIGINAL"]
    )
    df["TICKER_YF"] = df["TICKER_MAPEADO"] + ".SA"

    n_map = (df["TICKER_ORIGINAL"] != df["TICKER_MAPEADO"]).sum()
    n_est = df["ESTATAL"].sum()

    log.info("  → %d empresas | %d setores | %d remapeados | %d estatais",
             len(df), df["SETOR_B3"].nunique(), n_map, n_est)

    # Log das estatais para conferência
    for _, r in df[df["ESTATAL"]].iterrows():
        log.info("    [ESTATAL] %s — %s (%s)",
                 r["TICKER_ORIGINAL"], str(r["DENOM_SOCIAL"])[:45], r["SETOR_B3"])

    return df

In [340]:
import os
import time
from alpha_vantage.timeseries import TimeSeries

# Forma segura: defina como variável de ambiente (recomendado)
# No terminal: export ALPHA_VANTAGE_KEY=LB3KHS0AN1R2E36B (Linux/Mac) ou set no Windows
ALPHA_VANTAGE_KEY = os.getenv('ALPHA_VANTAGE_KEY')

# Fallback temporário só para teste local (remova em produção!)
if not ALPHA_VANTAGE_KEY:
    ALPHA_VANTAGE_KEY = 'SSU8PQU94ONCSLAF'  # Apague isso depois de configurar env
    print("AVISO: Usando key hardcoded – configure como env var para segurança!")

AVISO: Usando key hardcoded – configure como env var para segurança!


In [341]:
def baixar_precos_yfinance(tickers: list, start="2001-01-01", end="2023-12-31"):
    """Baixa preços ajustados e volumes via yfinance (primário) + Alpha Vantage (fallback)."""
    import yfinance as yf

    log.info("Baixando preços de %d tickers (primário: yfinance, fallback: Alpha Vantage)...", len(tickers))
    all_close = {}
    all_volume = {}
    diagnostico_list = []

    blocos = [tickers[i:i+50] for i in range(0, len(tickers), 50)]

    # Cliente Alpha Vantage (inicializa só se key válida)
    ts_av = None
    if ALPHA_VANTAGE_KEY:
        try:
            ts_av = TimeSeries(key=ALPHA_VANTAGE_KEY, output_format='pandas')
        except Exception as e:
            log.warning("Falha ao inicializar Alpha Vantage: %s", e)

    for idx, bloco in enumerate(blocos):
        log.info("  Bloco %d/%d (%d tickers)", idx+1, len(blocos), len(bloco))
        try:
            data = yf.download(bloco, start=start, end=end, auto_adjust=True,
                               progress=False, threads=True)
            if data.empty:
                raise ValueError("Dados vazios no yfinance")

            if isinstance(data.columns, pd.MultiIndex):
                close = data["Close"]
                volume = data["Volume"]
            else:
                close = data[["Close"]]; close.columns = bloco
                volume = data[["Volume"]]; volume.columns = bloco

            for col in close.columns:
                if close[col].notna().sum() > 0:
                    all_close[col] = close[col]
                    all_volume[col] = volume[col] if col in volume.columns else pd.Series(dtype=float)
                    diagnostico_list.append({
                        "ticker_yf": col, "status": "ok", "fonte": "yfinance", "motivo": ""
                    })
                else:
                    diagnostico_list.append({
                        "ticker_yf": col, "status": "falha", "fonte": "", "motivo": "no_data_yf"
                    })

            for t in bloco:
                if t not in close.columns:
                    diagnostico_list.append({
                        "ticker_yf": t, "status": "falha", "fonte": "", "motivo": "not_found_yf"
                    })

        except Exception as e:
            log.warning("  Erro yfinance bloco %d: %s → Marcando para fallback", idx+1, e)
            for t in bloco:
                diagnostico_list.append({
                    "ticker_yf": t, "status": "falha", "fonte": "", "motivo": f"erro_yf: {str(e)}"
                })

        time.sleep(0.3)

    # Fallback Alpha Vantage para falhas
    falhas = [d for d in diagnostico_list if d["status"] == "falha"]
    if falhas and ts_av:
        log.info("  Fallback Alpha Vantage para %d tickers falhos...", len(falhas))
        for diag in falhas:
            t = diag["ticker_yf"]
            try:
                data_av, _ = ts_av.get_daily(symbol=t, outputsize='full')
                data_av = data_av.rename(columns={
                    '1. open': 'Open', '2. high': 'High', '3. low': 'Low',
                    '4. close': 'Close', '5. volume': 'Volume'
                })
                if data_av['Close'].notna().sum() > 0:
                    all_close[t] = data_av['Close']
                    all_volume[t] = data_av['Volume']
                    diag["status"] = "ok"
                    diag["fonte"] = "alpha_vantage"
                    diag["motivo"] = ""
                
                time.sleep(12)  # Rate limit free (5 calls/min)

            except Exception as av_e:
                log.warning("  Falha Alpha Vantage %s: %s", t, av_e)
                diag["motivo"] += f"; erro_av: {str(av_e)}"

    # Finaliza
    df_precos = pd.DataFrame(all_close)
    df_precos.index = pd.to_datetime(df_precos.index)
    df_volumes = pd.DataFrame(all_volume)
    df_volumes.index = pd.to_datetime(df_volumes.index)

    df_diag = pd.DataFrame(diagnostico_list)
    n_ok = (df_diag["status"] == "ok").sum()
    n_falha = (df_diag["status"] == "falha").sum()

    log.info("  → OK: %d | Falha: %d (%.1f%%)", n_ok, n_falha,
             100 * n_falha / max(len(tickers), 1))

    return df_precos, df_volumes, df_diag

In [342]:
time.sleep(5)

In [343]:
def baixar_ibovespa(start="2000-01-02", end="2023-12-31"):
    import yfinance as yf
    log.info("Baixando Ibovespa ...")
    ibov = yf.download("^BVSP", start=start, end=end, auto_adjust=True, progress=False)
    serie = ibov["Close"].squeeze()
    serie.index = pd.to_datetime(serie.index)
    serie.name = "IBOV"
    log.info("  → %d obs", len(serie))
    return serie

In [344]:
def baixar_fatores_nefin():
    """
    Baixa fatores Fama-French brasileiros do NEFIN-USP (Versão CSV único).
    Retorna DataFrame com colunas: Mkt_Rf, SMB, HML, Rf.
    Se falhar, retorna None (script continua sem FF3).
    """
    log.info("Baixando fatores Fama-French do NEFIN-USP ...")
    
    # URL definida globalmente ou localmente
    url_csv = "https://nefin.com.br/resources/risk_factors/nefin_factors.csv"
    
    try:
        # Tenta ler o CSV direto da URL
        df = pd.read_csv(url_csv)

        # 1. Tratamento de Data (O CSV novo tem a coluna 'Date' pronta)
        if "Date" in df.columns:
            df["date"] = pd.to_datetime(df["Date"])
            df = df.set_index("date")
        else:
            # Fallback caso o formato mude para year/month/day
            if {"year", "month", "day"}.issubset(df.columns):
                 df["date"] = pd.to_datetime(df[["year", "month", "day"]])
                 df = df.set_index("date")
            else:
                log.warning("  Formato de data desconhecido no CSV do NEFIN.")
                return None

        # 2. Renomear colunas para o padrão do script (Mkt_Rf, SMB, HML, Rf)
        # O CSV vem como: Rm_minus_Rf, SMB, HML, Risk_Free
        rename_map = {
            "Rm_minus_Rf": "Mkt_Rf", 
            "Risk_Free": "Rf"
        }
        df = df.rename(columns=rename_map)

        # 3. Filtrar apenas as colunas necessárias
        cols_necessarias = ["Mkt_Rf", "SMB", "HML", "Rf"]
        
        # Verifica se todas existem
        if not set(cols_necessarias).issubset(df.columns):
            log.warning(f"  Colunas faltando no NEFIN. Encontradas: {df.columns.tolist()}")
            return None

        df_factors = df[cols_necessarias]

        log.info("  → Fatores NEFIN: %d obs (de %s a %s)",
                 len(df_factors), 
                 df_factors.index.min().strftime('%Y-%m'), 
                 df_factors.index.max().strftime('%Y-%m'))
        
        return df_factors

    except Exception as e:
        log.warning("  Falha ao baixar NEFIN: %s. FF3 desabilitado.", e)
        return None

In [345]:
# df_teste = baixar_fatores_nefin()

In [346]:
def aplicar_filtro_existencia(df_precos, df_empresas):
    """
    Invalida (NaN) precos anteriores a data de registro (DT_REG).
    Assume que DT_REG existe e que nao ha cancelamentos.
    """
    log.info("Aplicando filtro de existencia (apenas DT_REG)...")

    # Cria copia para nao alterar o original
    df = df_precos.copy()

    # Remove fuso horario para evitar erro de comparacao com o Excel
    if df.index.tz is not None:
        df.index = df.index.tz_localize(None)

    # Dicionario: Ticker -> Data Inicio
    lookup = {}
    col_inicio = "DT_REG"

    for _, row in df_empresas.iterrows():
        tk = row["TICKER_YF"]
        # Converte direto
        reg = pd.to_datetime(row[col_inicio], errors='coerce')

        # Se houver duplicata de ticker, preserva a data mais antiga (conservador)
        if tk in lookup:
            old_reg = lookup[tk]
            if pd.notna(reg) and (pd.isna(old_reg) or reg < old_reg):
                lookup[tk] = reg
        else:
            lookup[tk] = reg

    total_cortes = 0

    # Aplica o filtro coluna por coluna
    for col in df.columns:
        if col in lookup:
            dt_inicio = lookup[col]

            if pd.notna(dt_inicio):
                # Corta tudo que vier antes da data de registro
                mask = df.index < dt_inicio
                if mask.any():
                    qtd = df.loc[mask, col].notna().sum()
                    if qtd > 0:
                        total_cortes += qtd
                        df.loc[mask, col] = np.nan

    log.info("  -> Filtro aplicado. Observacoes removidas (pre-inicio): %d", total_cortes)
    
    return df

## ETAPA 2 – ÍNDICES SETORIAIS

In [347]:
# =============================================================================
# ETAPA 2 – ÍNDICES SETORIAIS (EW + VW) + FILTRO DE LIQUIDEZ
# =============================================================================
#
# Fluxo:
#   1. Calcula retornos log de cada ativo individual
#   2. Aplica filtro de liquidez por (ticker, ano): exige ≥80% de pregões
#   3. Constrói índices setoriais em dois esquemas:
#      - Equal-Weighted (EW): média simples dos retornos
#      - Volume-Weighted (VW): ponderado por volume financeiro médio 20d
#   4. Registra composição (N real de empresas por setor/ano após filtro)
#
# NOTA sobre retornos log:
#   Usamos r_t = ln(P_t / P_{t-1}). A média EW de retornos log não é
#   exatamente o retorno de um portfólio equal-weighted (que seria média
#   de retornos aritméticos). Para janelas curtas ([-5,+5]) a diferença
#   é desprezível. Para janelas de 6 meses, pode importar marginalmente.
#   A maioria dos event studies (MacKinlay 1997, Silva et al. 2015) usa
#   retornos log. Documentamos aqui por transparência.
# =============================================================================

In [348]:
# ============================================================================
# ETAPA 2 – ÍNDICES SETORIAIS (Versão Aprimorada)
# ============================================================================

def aplicar_winsorizacao(df_ret, lower=0.01, upper=0.99):
    """Winsorização: trata outliers nos retornos (1% e 99%)."""
    log.info(f"Aplicando winsorização ({lower*100:.0f}% / {upper*100:.0f}%)...")
    df_wins = df_ret.copy()
    for col in df_wins.columns:
        q_low = df_wins[col].quantile(lower)
        q_high = df_wins[col].quantile(upper)
        df_wins[col] = df_wins[col].clip(lower=q_low, upper=q_high)
    return df_wins


def aplicar_filtro_penny(df_precos, ano, min_preco=1.0):
    """Remove penny stocks: preço médio no ano < R$ 1,00."""
    mask = df_precos.index.year == ano
    precos_ano = df_precos.loc[mask]
    if precos_ano.empty:
        return []
    media_preco = precos_ano.mean()
    return media_preco[media_preco >= min_preco].index.tolist()


def aplicar_filtro_liquidez(df_ret, ano, min_pct=MIN_PREGOES_PCT):
    """Filtro de liquidez: mínimo % de pregões no ano."""
    mask_ano = df_ret.index.year == ano
    ret_ano = df_ret.loc[mask_ano]
    if ret_ano.empty:
        return []
    n_pregoes = mask_ano.sum()
    min_obs = int(n_pregoes * min_pct)
    obs_por_ticker = ret_ano.notna().sum()
    return obs_por_ticker[obs_por_ticker >= min_obs].index.tolist()


def construir_indices_setoriais(df_precos, df_volumes, df_empresas, aplicar_wins=True):
    log.info("ETAPA 2: ÍNDICES SETORIAIS (EW + VW) + TRATAMENTO ESPECIAL PARA PETRÓLEO")

    ret = np.log(df_precos / df_precos.shift(1))
    if aplicar_wins:
        ret = aplicar_winsorizacao(ret)

    vol_fin = df_precos * df_volumes
    t2s = {row["TICKER_YF"]: row["SETOR_B3"] for _, row in df_empresas.iterrows()}

    series_ew = {}
    series_vw = {}
    composicao = []

    for setor in SETORES_B3:
        cols_setor = [c for c in ret.columns if t2s.get(c) == setor]
        if not cols_setor:
            continue

        ew_parts = []
        vw_parts = []

        for ano in range(2001, 2024):
            mask_ano = ret.index.year == ano
            if mask_ano.sum() == 0:
                continue

            # === TRATAMENTO ESPECIAL PARA PETRÓLEO ===
            if setor in SETORES_ESPECIAIS and ano in TICKERS_PETROLEO_FORCADOS:
                cols_ano = [t for t in TICKERS_PETROLEO_FORCADOS[ano] if t in ret.columns]
                log.info(f"  → Forçando inclusão manual de {len(cols_ano)} tickers de Petróleo em {ano}")
            else:
                # Filtros normais para outros setores
                ok_liquidez = set(aplicar_filtro_liquidez(ret, ano, MIN_PREGOES_PCT))
                ok_penny = set(aplicar_filtro_penny(df_precos, ano))
                cols_ano = [c for c in cols_setor if c in ok_liquidez and c in ok_penny]

            composicao.append({
                "setor": setor,
                "ano": ano,
                "n_tickers_setor": len(cols_setor),
                "n_com_dados_liquidos": len(cols_ano),
                "filtro_removeu": len(cols_setor) - len(cols_ano),
            })

            min_empresas = MIN_EMPRESAS_SETOR_ESPECIAL if setor in SETORES_ESPECIAIS else MIN_EMPRESAS_SETOR
            if len(cols_ano) < min_empresas:
                continue

            ret_ano = ret.loc[mask_ano, cols_ano]

            # EW
            ew_parts.append(ret_ano.mean(axis=1))

            # VW (mais importante para Petróleo)
            vf_ano = vol_fin.loc[mask_ano, cols_ano].rolling(20, min_periods=5).mean()
            vf_sum = vf_ano.sum(axis=1)
            weights = vf_ano.div(vf_sum.replace(0, np.nan), axis=0)
            vw_parts.append((ret_ano * weights).sum(axis=1))

        if ew_parts:
            series_ew[setor] = pd.concat(ew_parts).sort_index()
        if vw_parts:
            series_vw[setor] = pd.concat(vw_parts).sort_index()

        # Log mais claro
        comp_setor = [c for c in composicao if c["setor"] == setor]
        anos_validos = sum(1 for c in comp_setor if c["n_com_dados_liquidos"] >= min_empresas)
        log.info(f"  {setor:35} → {len(cols_setor):2d} tickers | {anos_validos:2d}/{len(comp_setor)} anos válidos")

    df_ret_ew = pd.DataFrame(series_ew)
    df_ret_vw = pd.DataFrame(series_vw)
    df_comp = pd.DataFrame(composicao)

    return df_ret_ew, df_ret_vw, df_comp

# Função auxiliar para janela de estimação
def janela_estimacao(bdates, dt_evento):
    """Janela de estimação padrão: [-252, -30] dias úteis."""
    return (
        offset_du(bdates, dt_evento, ESTIMACAO_INICIO_DU),
        offset_du(bdates, dt_evento, ESTIMACAO_FIM_DU)
    )


# Tabela de Sobrevivência (corrigida para seu Excel sem DT_CANCEL)
def gerar_tabela_sobrevivencia(df_empresas, df_precos):
    """Tabela de cobertura: N esperado vs N com dados por setor/ano."""
    log.info("Gerando tabela de sobrevivência...")
    
    t2s = {row["TICKER_YF"]: row["SETOR_B3"] for _, row in df_empresas.iterrows()}
    registros = []

    for ano in range(2001, 2024):
        mask_ano = df_precos.index.year == ano
        if mask_ano.sum() == 0:
            continue
        
        precos_ano = df_precos.loc[mask_ano]

        for setor in SETORES_B3:
            cols = [c for c in precos_ano.columns if t2s.get(c) == setor]
            n_dados = precos_ano[cols].notna().any().sum() if cols else 0

            # Como não tem mais DT_CANCEL, usamos apenas DT_REG
            n_esp = (
                (df_empresas["SETOR_B3"] == setor) &
                (df_empresas["DT_REG"].dt.year <= ano)
            ).sum()

            cobertura = round(100 * n_dados / max(n_esp, 1), 1)

            registros.append({
                "ano": ano,
                "setor": setor,
                "n_esperado": n_esp,
                "n_com_dados": n_dados,
                "cobertura_pct": cobertura,
            })

    df_sobrev = pd.DataFrame(registros)
    return df_sobrev

## ETAPA 3 – JANELAS DE EVENTO (VERSÃO FINAL SIMPLES E SEGURA)


In [349]:
# ============================================================================
# ETAPA 3 – JANELAS DE EVENTO (VERSÃO FINAL SIMPLES E SEGURA)
# ============================================================================

def offset_du(bdates, data_ref, offset=0):
    """Retorna a data útil mais próxima com offset."""
    bdays = bdates.sort_values()
    pos = min(bdays.searchsorted(data_ref), len(bdays) - 1)
    target = max(0, min(pos + offset, len(bdays) - 1))
    return bdays[target]


def coletar_todas_janelas(bdates, ano):
    """
    Coleta todas as janelas ativas para um ano eleitoral.
    Versão final: força o fim das janelas semestrais/estendida em dezembro do ano correto.
    """
    dt1 = DATAS_PRIMEIRO_TURNO[ano]
    dt2 = DATAS_SEGUNDO_TURNO.get(ano)

    jans = {}

    # 1. Janelas do 1º Turno
    for nome, (di, df_) in JANELAS_EVENTO_1T.items():
        jans[nome] = (
            offset_du(bdates, dt1, di),
            offset_du(bdates, dt1, df_)
        )

    # 2. Janelas do 2º Turno
    if dt2:
        for nome, (di, df_) in JANELAS_EVENTO_2T.items():
            jans[nome] = (
                offset_du(bdates, dt2, di),
                offset_du(bdates, dt2, df_)
            )

    # 3. Janela Entre Turnos
    if JANELA_ENTRE_TURNOS and dt2:
        jans["entre_turnos"] = (
            offset_du(bdates, dt1, -5),
            offset_du(bdates, dt2, -1)
        )

    # 4. Janelas Semestrais (corrigido com força)
    if JANELAS_CICLO:
        jans["ciclo_1sem"] = (
            offset_du(bdates, pd.Timestamp(f"{ano}-01-01"), 0),
            offset_du(bdates, pd.Timestamp(f"{ano}-06-30"), 0)
        )
        jans["ciclo_2sem"] = (
            offset_du(bdates, pd.Timestamp(f"{ano}-07-01"), 0),
            offset_du(bdates, pd.Timestamp(f"{ano+1}-01-01"), -1)   # Força último dia útil de dezembro
        )

    # 5. Janela Estendida (corrigido)
    if JANELA_ESTENDIDA:
        jans["estendida"] = (
            offset_du(bdates, pd.Timestamp(f"{ano}-07-01"), 0),
            offset_du(bdates, pd.Timestamp(f"{ano+1}-01-01"), -1)   # Força fim em dezembro
        )

    # 6. Janelas-Espelho (T-1)
    if JANELAS_ESPELHO:
        principais = {k: v for k, v in jans.items() if not k.startswith("espelho_")}
        for nome, (ini, fim) in principais.items():
            try:
                ini_esp = ini - pd.DateOffset(years=1)
                fim_esp = fim - pd.DateOffset(years=1)
                jans[f"espelho_{nome}"] = (
                    offset_du(bdates, ini_esp, 0),
                    offset_du(bdates, fim_esp, 0)
                )
            except:
                pass

    return jans


def imprimir_janelas(jans, ano):
    print(f"\n=== Janelas definidas para {ano} ===")
    for nome, (ini, fim) in sorted(jans.items()):
        dias = (fim - ini).days
        print(f"  {nome:25} → {ini.date()} até {fim.date()} ({dias:3d} dias)")

In [350]:
bdates = pd.date_range("2000-01-01", "2023-12-31", freq='B')
jans = coletar_todas_janelas(bdates, 2022)
imprimir_janelas(jans, 2022)


=== Janelas definidas para 2022 ===
  antecipacao_45            → 2022-08-01 até 2022-09-30 ( 60 dias)
  antecipacao_60            → 2022-07-11 até 2022-09-30 ( 81 dias)
  ciclo_1sem                → 2022-01-03 até 2022-06-30 (178 dias)
  ciclo_2sem                → 2022-07-01 até 2022-12-30 (182 dias)
  entre_turnos              → 2022-09-26 até 2022-10-28 ( 32 dias)
  espelho_antecipacao_45    → 2021-08-02 até 2021-09-30 ( 59 dias)
  espelho_antecipacao_60    → 2021-07-12 até 2021-09-30 ( 80 dias)
  espelho_ciclo_1sem        → 2021-01-04 até 2021-06-30 (177 dias)
  espelho_ciclo_2sem        → 2021-07-01 até 2021-12-30 (182 dias)
  espelho_entre_turnos      → 2021-09-27 até 2021-10-28 ( 31 dias)
  espelho_estendida         → 2021-07-01 até 2021-12-30 (182 dias)
  espelho_reacao_ampla_1t   → 2021-09-06 até 2021-11-01 ( 56 dias)
  espelho_reacao_curta_1t   → 2021-09-27 até 2021-10-11 ( 14 dias)
  espelho_reacao_curta_2t   → 2021-10-25 até 2021-11-08 ( 14 dias)
  espelho_reacao_media_1t

## ============================================================================
## ETAPA 4 – MODELOS DE ESTIMAÇÃO E INFERÊNCIA ESTATÍSTICA
## ============================================================================

In [351]:
def estimar_ols_simples(ret_y, ret_x):
    """OLS simples: R_i = α + β·R_m + ε (Modelo de Mercado)"""
    df = pd.DataFrame({"y": ret_y, "x": ret_x}).dropna()
    if len(df) < 30:
        return None
    X = np.column_stack([np.ones(len(df)), df["x"].values])
    Y = df["y"].values
    try:
        coef = np.linalg.lstsq(X, Y, rcond=None)[0]
        resid = Y - X @ coef
        ss_res = (resid**2).sum()
        ss_tot = ((Y - Y.mean())**2).sum()
        return {
            "alpha": coef[0],
            "beta": coef[1],
            "sigma_resid": np.sqrt(ss_res / (len(df) - 2)),
            "r_squared": 1 - ss_res / ss_tot if ss_tot > 0 else 0,
            "n_obs_est": len(df)
        }
    except:
        return None

In [352]:
def estimar_ols_hac(ret_y, ret_x, maxlags=1):
    """OLS com erros padrão HAC (Newey-West) — mais robusto"""
    import statsmodels.api as sm
    df = pd.DataFrame({"y": ret_y, "x": ret_x}).dropna()
    if len(df) < 30:
        return None
    X = sm.add_constant(df["x"])
    try:
        model = sm.OLS(df["y"], X).fit(cov_type="HAC", cov_kwds={"maxlags": maxlags})
        return {
            "alpha": model.params.iloc[0],
            "beta": model.params.iloc[1],
            "alpha_pv": model.pvalues.iloc[0],
            "beta_pv": model.pvalues.iloc[1],
            "sigma_resid": np.sqrt(model.mse_resid),
            "r_squared": model.rsquared,
            "n_obs_est": int(model.nobs)
        }
    except:
        return None

In [353]:
def estimar_ff3(ret_y, df_factors_window):
    """Fama-French 3 fatores (NEFIN)"""
    import statsmodels.api as sm
    common = ret_y.index.intersection(df_factors_window.index)
    if len(common) < 30:
        return None
    y = ret_y.loc[common]
    fac = df_factors_window.loc[common]
    y_excess = y - fac.get("Rf", 0)
    X_cols = [c for c in ["Mkt_Rf", "SMB", "HML"] if c in fac.columns]
    if len(X_cols) < 2:
        return None
    X = sm.add_constant(fac[X_cols])
    try:
        model = sm.OLS(y_excess, X).fit(cov_type="HAC", cov_kwds={"maxlags": 1})
        return {
            "alpha_ff3": model.params.iloc[0],
            "beta_mkt": model.params.get("Mkt_Rf", np.nan),
            "beta_smb": model.params.get("SMB", np.nan),
            "beta_hml": model.params.get("HML", np.nan),
            "alpha_pv_ff3": model.pvalues.iloc[0],
            "sigma_resid_ff3": np.sqrt(model.mse_resid),
            "r_squared_ff3": model.rsquared,
            "n_obs_est_ff3": int(model.nobs)
        }
    except:
        return None


def calcular_ar(ret_y, ret_x, alpha, beta):
    """Calcula Retornos Anormais (AR) - Modelo de Mercado"""
    df = pd.DataFrame({"y": ret_y, "x": ret_x}).dropna()
    return df["y"] - (alpha + beta * df["x"])


def calcular_ar_ff3(ret_y, df_factors_window, params_ff3):
    """Calcula Retornos Anormais - Fama-French 3"""
    common = ret_y.index.intersection(df_factors_window.index)
    if len(common) == 0:
        return pd.Series(dtype=float)
    y = ret_y.loc[common]
    fac = df_factors_window.loc[common]
    rf = fac.get("Rf", 0)
    expected = (params_ff3["alpha_ff3"] +
                params_ff3.get("beta_mkt", 0) * fac.get("Mkt_Rf", 0) +
                params_ff3.get("beta_smb", 0) * fac.get("SMB", 0) +
                params_ff3.get("beta_hml", 0) * fac.get("HML", 0))
    return (y - rf) - expected


def tstat_car_silva(car_values, ar_series_list, n_dias):
    """Teste t corrigido por autocorrelação (Silva et al., 2015)"""
    n = len(car_values)
    if n < 2:
        return {"t_silva": np.nan, "p_silva": np.nan}
    car_mean = np.mean(car_values)
    variances, covariances = [], []
    for ar_s in ar_series_list:
        arr = np.array(ar_s.dropna()) if hasattr(ar_s, 'dropna') else np.array(ar_s)
        if len(arr) < 2: continue
        variances.append(np.var(arr, ddof=1))
        if len(arr) > 2:
            covariances.append(np.cov(arr[:-1], arr[1:])[0, 1])
    if not variances:
        return {"t_silva": np.nan, "p_silva": np.nan}
    t = max(n_dias, 1)
    csd_sq = t * np.mean(variances) + 2 * max(t-1, 0) * (np.mean(covariances) if covariances else 0)
    if csd_sq <= 0:
        return {"t_silva": np.nan, "p_silva": np.nan}
    t_stat = car_mean * np.sqrt(n) / np.sqrt(csd_sq)
    p_val = 2 * (1 - stats.t.cdf(abs(t_stat), max(n-1, 1)))
    return {"t_silva": t_stat, "p_silva": p_val}


def tstat_car_bmp(car_values, sigma_resids, n_dias):
    """Teste BMP (1991) - controla variância induzida pelo evento"""
    n = len(car_values)
    if n < 2:
        return {"t_bmp": np.nan, "p_bmp": np.nan}
    denom = sigma_resids * np.sqrt(max(n_dias, 1))
    scar = np.where(denom > 0, car_values / denom, np.nan)
    scar = scar[~np.isnan(scar)]
    if len(scar) < 2:
        return {"t_bmp": np.nan, "p_bmp": np.nan}
    s_std = np.std(scar, ddof=1)
    if s_std == 0:
        return {"t_bmp": np.nan, "p_bmp": np.nan}
    t_bmp = np.mean(scar) * np.sqrt(len(scar)) / s_std
    p_bmp = 2 * (1 - stats.t.cdf(abs(t_bmp), len(scar)-1))
    return {"t_bmp": t_bmp, "p_bmp": p_bmp}

## ETAPA 5 – EXECUÇÃO DO EVENT STUDY (MODELO DE MERCADO)

In [354]:
# ============================================================================
# ETAPA 5 – EXECUÇÃO DO EVENT STUDY (MODELO DE MERCADO)
# ============================================================================

def executar_analise(df_ret_setores, ret_ibov, cenario="completo", ponderacao="EW"):
    """
    Executa o Event Study completo para todos os anos e setores.
    
    Parâmetros:
        df_ret_setores : DataFrame de retornos setoriais (EW ou VW)
        ret_ibov       : Série de retornos do Ibovespa
        cenario        : "completo" ou "sem_crises"
        ponderacao     : "EW" ou "VW"
    
    Retorna:
        DataFrame com todos os resultados (CAR, testes estatísticos, etc.)
    """
    log.info("=" * 70)
    log.info(f"ETAPA 5: EXECUTANDO EVENT STUDY → {cenario.upper()} | {ponderacao}")
    log.info("=" * 70)

    bdates = df_ret_setores.dropna(how="all").index
    resultados = []

    for ano in ANOS_ELEITORAIS:
        log.info(f"  Processando ano eleitoral: {ano}")
        
        dt1 = DATAS_PRIMEIRO_TURNO[ano]
        est_ini, est_fim = janela_estimacao(bdates, dt1)
        
        # Coleta todas as janelas (incluindo entre turnos, ciclo, estendida, espelhos)
        jans = coletar_todas_janelas(bdates, ano)

        for setor in df_ret_setores.columns:
            # Janela de estimação
            m_est = (df_ret_setores.index >= est_ini) & (df_ret_setores.index <= est_fim)
            rs_est = df_ret_setores.loc[m_est, setor].dropna()
            rm_est = ret_ibov.loc[m_est].dropna()

            # Estima modelo
            params = estimar_ols_simples(rs_est, rm_est)
            if params is None:
                continue

            params_hac = estimar_ols_hac(rs_est, rm_est)

            # Para cada janela de evento
            for nome_j, (ji, jf) in jans.items():
                m_j = (df_ret_setores.index >= ji) & (df_ret_setores.index <= jf)
                rs_j = df_ret_setores.loc[m_j, setor].dropna()
                rm_j = ret_ibov.loc[m_j].dropna()

                if len(rs_j) < 3:
                    continue

                ar = calcular_ar(rs_j, rm_j, params["alpha"], params["beta"])
                car = ar.sum()
                nd = len(ar)
                ar_std = ar.std(ddof=1) if len(ar) > 1 else np.nan

                t_simples = (ar.mean() * np.sqrt(nd) / ar_std) if ar_std > 0 else np.nan
                p_simples = 2 * (1 - stats.t.cdf(abs(t_simples), max(nd-1, 1))) if not np.isnan(t_simples) else np.nan

                resultados.append({
                    "cenario": cenario,
                    "ponderacao": ponderacao,
                    "ano": ano,
                    "setor": setor,
                    "janela": nome_j,
                    "car": car,
                    "ar_medio": ar.mean(),
                    "ar_std": ar_std,
                    "n_dias": nd,
                    "alpha": params["alpha"],
                    "beta": params["beta"],
                    "r_squared": params["r_squared"],
                    "sigma_resid": params["sigma_resid"],
                    "n_obs_est": params["n_obs_est"],
                    "alpha_hac_pv": params_hac["alpha_pv"] if params_hac else np.nan,
                    "beta_hac_pv": params_hac["beta_pv"] if params_hac else np.nan,
                    "est_inicio": est_ini,
                    "est_fim": est_fim,
                    "janela_inicio": ji,
                    "janela_fim": jf,
                    "t_simples": t_simples,
                    "p_simples": p_simples,
                    "t_silva": np.nan,
                    "p_silva": np.nan,
                    "t_bmp": np.nan,
                    "p_bmp": np.nan,
                })

    df_res = pd.DataFrame(resultados)

    # === Testes cross-sectionais (por ano e janela) ===
    if not df_res.empty:
        log.info("  Calculando testes cross-sectionais (Silva e BMP)...")
        for (ano, jan), grp in df_res.groupby(["ano", "janela"]):
            cars = grp["car"].values
            sigmas = grp["sigma_resid"].values
            nd = int(grp["n_dias"].median())

            ar_lists = []
            for _, row in grp.iterrows():
                m = (df_ret_setores.index >= row["janela_inicio"]) & \
                    (df_ret_setores.index <= row["janela_fim"])
                rs = df_ret_setores.loc[m, row["setor"]].dropna()
                rm = ret_ibov.loc[m].dropna()
                ar_lists.append(calcular_ar(rs, rm, row["alpha"], row["beta"]))

            silva = tstat_car_silva(cars, ar_lists, nd)
            bmp = tstat_car_bmp(cars, sigmas, nd)

            mask = (df_res["ano"] == ano) & (df_res["janela"] == jan) & \
                   (df_res["cenario"] == cenario) & (df_res["ponderacao"] == ponderacao)

            df_res.loc[mask, "t_silva"] = silva["t_silva"]
            df_res.loc[mask, "p_silva"] = silva["p_silva"]
            df_res.loc[mask, "t_bmp"] = bmp["t_bmp"]
            df_res.loc[mask, "p_bmp"] = bmp["p_bmp"]

    # Flags de significância
    for col, pv in [("sig5_simples", "p_simples"), ("sig5_silva", "p_silva"), ("sig5_bmp", "p_bmp")]:
        df_res[col] = df_res[pv] < 0.05

    for col, pv in [("sig10_simples", "p_simples"), ("sig10_silva", "p_silva"), ("sig10_bmp", "p_bmp")]:
        df_res[col] = df_res[pv] < 0.10

    log.info(f"  → Event Study finalizado: {len(df_res)} linhas geradas")
    return df_res

## ETAPA 5b – EVENT STUDY COM FAMA-FRENCH 3 FATORES (NEFIN)

In [355]:
# ============================================================================
# ETAPA 5b – EVENT STUDY COM FAMA-FRENCH 3 FATORES (CORRIGIDO)
# ============================================================================

def executar_analise_ff3(df_ret_setores, df_factors, cenario="completo", ponderacao="EW"):
    """
    Executa Event Study com Fama-French 3 fatores (NEFIN).
    Versão corrigida com alinhamento de índices.
    """
    if df_factors is None or df_factors.empty:
        log.warning("  FF3 desabilitado (fatores NEFIN não disponíveis)")
        return pd.DataFrame()

    log.info(f"  EVENT STUDY FF3 → {cenario.upper()} | {ponderacao}")

    bdates = df_ret_setores.dropna(how="all").index
    resultados = []

    for ano in ANOS_ELEITORAIS:
        dt1 = DATAS_PRIMEIRO_TURNO[ano]
        est_ini, est_fim = janela_estimacao(bdates, dt1)

        # Janelas principais para FF3
        jans = coletar_todas_janelas(bdates, ano)
        jans_ff3 = {k: v for k, v in jans.items() 
                   if k in ["antecipacao_45", "reacao_curta_1t", "reacao_curta_2t", 
                            "reacao_media_1t", "entre_turnos"]}

        # Alinhamento correto: intersectar índices primeiro
        common_idx = df_ret_setores.index.intersection(df_factors.index)
        df_ret_aligned = df_ret_setores.loc[common_idx]
        df_fac_aligned = df_factors.loc[common_idx]

        for setor in df_ret_aligned.columns:
            # Janela de estimação alinhada
            m_est = (df_fac_aligned.index >= est_ini) & (df_fac_aligned.index <= est_fim)
            rs_est = df_ret_aligned.loc[m_est, setor].dropna()
            fac_est = df_fac_aligned.loc[m_est]

            params_ff3 = estimar_ff3(rs_est, fac_est)
            if params_ff3 is None:
                continue

            for nome_j, (ji, jf) in jans_ff3.items():
                m_j = (df_ret_aligned.index >= ji) & (df_ret_aligned.index <= jf)
                rs_j = df_ret_aligned.loc[m_j, setor].dropna()
                fac_j = df_fac_aligned.loc[m_j]

                if len(rs_j) < 3:
                    continue

                ar = calcular_ar_ff3(rs_j, fac_j, params_ff3)
                if len(ar) < 3:
                    continue

                car = ar.sum()
                nd = len(ar)
                ar_std = ar.std() if len(ar) > 1 else np.nan
                t_s = (ar.mean() * np.sqrt(nd) / ar_std) if ar_std > 0 else np.nan
                p_s = 2 * (1 - stats.t.cdf(abs(t_s), max(nd-1, 1))) if not np.isnan(t_s) else np.nan

                resultados.append({
                    "cenario": cenario,
                    "ponderacao": ponderacao,
                    "modelo": "FF3",
                    "ano": ano,
                    "setor": setor,
                    "janela": nome_j,
                    "car": car,
                    "ar_medio": ar.mean(),
                    "n_dias": nd,
                    "alpha_ff3": params_ff3["alpha_ff3"],
                    "beta_mkt": params_ff3.get("beta_mkt"),
                    "beta_smb": params_ff3.get("beta_smb"),
                    "beta_hml": params_ff3.get("beta_hml"),
                    "r_squared": params_ff3.get("r_squared_ff3"),
                    "t_simples": t_s,
                    "p_simples": p_s,
                    "sig5": p_s < 0.05 if not np.isnan(p_s) else False,
                })

    df_ff3 = pd.DataFrame(resultados)
    log.info(f"  → FF3 finalizado: {len(df_ff3)} linhas")
    return df_ff3

## ETAPA 6 – TESTE PLACEBO (1000 pseudo-eventos)

In [356]:
# ============================================================================
# ETAPA 6 – TESTE PLACEBO
# ============================================================================

def executar_teste_placebo(df_ret, ret_ibov, n_placebos=N_PLACEBO_EVENTS, seed=42):
    """
    Teste Placebo: gera eventos fictícios em anos não-eleitorais.
    """
    log.info("=" * 70)
    log.info(f"ETAPA 6: TESTE PLACEBO ({n_placebos} pseudo-eventos)")
    log.info("=" * 70)

    np.random.seed(seed)
    bdates = df_ret.dropna(how="all").index
    anos_nao_eleitorais = [a for a in range(2003, 2023) if a not in ANOS_ELEITORAIS]

    resultados = []

    for i in range(n_placebos):
        ano_fake = np.random.choice(anos_nao_eleitorais)
        dt_fake = pd.Timestamp(f"{ano_fake}-10-05")   # data arbitrária

        est_ini, est_fim = janela_estimacao(bdates, dt_fake)

        for setor in df_ret.columns:
            m_est = (df_ret.index >= est_ini) & (df_ret.index <= est_fim)
            params = estimar_ols_simples(df_ret.loc[m_est, setor].dropna(), 
                                         ret_ibov.loc[m_est].dropna())
            if params is None:
                continue

            # Janelas placebo
            for nome, offset_i, offset_f in [
                ("placebo_antecip", -45, -1),
                ("placebo_reacao", -5, +5)
            ]:
                ji = offset_du(bdates, dt_fake, offset_i)
                jf = offset_du(bdates, dt_fake, offset_f)

                m_j = (df_ret.index >= ji) & (df_ret.index <= jf)
                rs = df_ret.loc[m_j, setor].dropna()
                rm = ret_ibov.loc[m_j].dropna()

                if len(rs) < 3:
                    continue

                ar = calcular_ar(rs, rm, params["alpha"], params["beta"])
                resultados.append({
                    "placebo_id": i,
                    "ano_placebo": ano_fake,
                    "setor": setor,
                    "janela": nome,
                    "car": ar.sum(),
                    "n_dias": len(ar)
                })

    df_placebo = pd.DataFrame(resultados)
    log.info(f"  → Teste Placebo finalizado: {len(df_placebo)} observações")
    return df_placebo

## ETAPA 6b – DIFFERENCE-IN-DIFFERENCES (Estatais × Privadas)

In [357]:
# ============================================================================
# ETAPA 6b – DIFFERENCE-IN-DIFFERENCES (Versão Corrigida)
# ============================================================================

def executar_did_empresas(df_precos, df_empresas, ret_ibov):
    """
    DiD no nível individual: CAR_estatais - CAR_privadas
    Versão corrigida com alinhamento explícito de índices.
    """
    log.info("=" * 70)
    log.info("ETAPA 6b: DIFFERENCE-IN-DIFFERENCES (Estatais vs Privadas)")
    log.info("=" * 70)

    # Retornos dos ativos individuais
    ret_ativos = np.log(df_precos / df_precos.shift(1))

    # === ALINHAMENTO GLOBAL (IMPORTANTE) ===
    common_idx = ret_ativos.index.intersection(ret_ibov.index)
    ret_ativos = ret_ativos.loc[common_idx]
    ret_ibov = ret_ibov.loc[common_idx]

    # Mapeamento ticker → informação
    t2info = {}
    for _, row in df_empresas.iterrows():
        tk = row["TICKER_YF"]
        t2info[tk] = {
            "setor": row["SETOR_B3"],
            "estatal": row["ESTATAL"],
            "denom": row.get("DENOM_SOCIAL", "")
        }

    bdates = ret_ativos.index
    janelas_did = ["antecipacao_45", "reacao_curta_1t", "reacao_curta_2t"]

    resultados = []

    for ano in ANOS_ELEITORAIS:
        dt1 = DATAS_PRIMEIRO_TURNO[ano]
        est_ini, est_fim = janela_estimacao(bdates, dt1)
        jans = coletar_todas_janelas(bdates, ano)
        jans_did = {k: v for k, v in jans.items() if k in janelas_did}

        for ticker in ret_ativos.columns:
            if ticker not in t2info:
                continue
            info = t2info[ticker]

            # Janela de estimação
            m_est = (ret_ativos.index >= est_ini) & (ret_ativos.index <= est_fim)
            params = estimar_ols_simples(ret_ativos.loc[m_est, ticker].dropna(),
                                         ret_ibov.loc[m_est].dropna())
            if params is None:
                continue

            for nome_j, (ji, jf) in jans_did.items():
                m_j = (ret_ativos.index >= ji) & (ret_ativos.index <= jf)
                ar = calcular_ar(ret_ativos.loc[m_j, ticker].dropna(),
                                 ret_ibov.loc[m_j].dropna(),
                                 params["alpha"], params["beta"])
                if len(ar) < 3:
                    continue

                resultados.append({
                    "ano": ano,
                    "janela": nome_j,
                    "ticker": ticker,
                    "setor": info["setor"],
                    "estatal": info["estatal"],
                    "car": ar.sum(),
                    "n_dias": len(ar)
                })

    df_did = pd.DataFrame(resultados)

    # Agregação
    agg = []
    for (ano, janela), grp in df_did.groupby(["ano", "janela"]):
        est = grp[grp["estatal"]]["car"].values
        priv = grp[~grp["estatal"]]["car"].values
        if len(est) < 2 or len(priv) < 2:
            continue
        delta = np.mean(est) - np.mean(priv)
        t, p = stats.ttest_ind(est, priv, equal_var=False)
        agg.append({
            "ano": ano,
            "janela": janela,
            "car_estatais": np.mean(est),
            "car_privadas": np.mean(priv),
            "delta_did": delta,
            "t_did": t,
            "p_did": p,
            "sig5_did": p < 0.05 if not np.isnan(p) else False,
            "n_estatais": len(est),
            "n_privadas": len(priv)
        })

    df_did_agg = pd.DataFrame(agg)

    log.info(f"  → DiD concluído: {len(df_did)} linhas individuais | {len(df_did_agg)} agregadas")
    return df_did, df_did_agg

## ETAPA 7 – VOLATILIDADE COMPARATIVA E SHARPE RATIO

In [358]:
# ============================================================================
# ETAPA 7 – VOLATILIDADE E SHARPE RATIO
# ============================================================================

def calcular_volatilidade_comparativa(df_ret):
    """Volatilidade anualizada: Eleitoral vs Não-Eleitoral."""
    log.info("Calculando volatilidade comparativa...")
    registros = []
    for setor in df_ret.columns:
        for ano in range(2002, 2023):
            r = df_ret.loc[df_ret.index.year == ano, setor].dropna()
            if len(r) < 20:
                continue
            registros.append({
                "setor": setor,
                "ano": ano,
                "tipo_ano": "Eleitoral" if ano in ANOS_ELEITORAIS else "Não-Eleitoral",
                "vol_anualizada": r.std() * np.sqrt(252)
            })
    return pd.DataFrame(registros)


def calcular_sharpe_comparativo(df_ret):
    """Sharpe Ratio anualizado comparando anos eleitorais vs não-eleitorais."""
    log.info("Calculando Sharpe Ratio comparativo...")
    registros = []
    for setor in df_ret.columns:
        for ano in range(2002, 2023):
            r = df_ret.loc[df_ret.index.year == ano, setor].dropna()
            if len(r) < 20:
                continue
            rf = SELIC_DIARIA.get(ano, 0.0003)
            excess = r - rf
            sharpe = (excess.mean() / excess.std() * np.sqrt(252)) if excess.std() > 0 else np.nan
            t, p = stats.ttest_1samp(excess.dropna(), 0)
            registros.append({
                "setor": setor,
                "ano": ano,
                "tipo_ano": "Eleitoral" if ano in ANOS_ELEITORAIS else "Não-Eleitoral",
                "sharpe_anualizado": sharpe,
                "excesso_medio": excess.mean(),
                "t_excesso": t,
                "p_excesso": p
            })
    return pd.DataFrame(registros)


def adicionar_benchmarking(df_res):
    """Adiciona comparação com Selic e IPCA proporcional."""
    df = df_res.copy()
    df["selic_prop"] = df["ano"].map(SELIC_ANUAL) * df["n_dias"] / 252
    df["ipca_prop"] = df["ano"].map(IPCA_ANUAL) * df["n_dias"] / 252
    df["car_exc_selic"] = df["car"] - df["selic_prop"]
    df["car_exc_ipca"] = df["car"] - df["ipca_prop"]
    return df

## ETAPA 8 – VISUALIZAÇÕES

In [359]:
# ============================================================================
# ETAPA 8 – VISUALIZAÇÕES (Versão Corrigida)
# ============================================================================

def gerar_visualizacoes(df_res, df_vol, df_placebo, df_did_agg, df_sharpe, output_dir):
    """
    Gera todas as visualizações principais do estudo.
    """
    import matplotlib
    matplotlib.use("Agg")
    import matplotlib.pyplot as plt
    import seaborn as sns

    sns.set_style("whitegrid")
    
    # Correção aqui: "figure.dpi" em vez de "dpi"
    plt.rcParams.update({
        "figure.figsize": (16, 9),
        "font.size": 12,
        "figure.dpi": 300
    })

    df = df_res[(df_res["cenario"] == "completo") & (df_res["ponderacao"] == "EW")]
    df_main = df[~df["janela"].str.startswith("espelho_")]

    output_dir = os.path.join(output_dir, "graficos")
    os.makedirs(output_dir, exist_ok=True)

    log.info("Gerando visualizações...")

    # 1. HEATMAPS
    hm_map = {
        "antecipacao_45": "Antecipação (−45 a −1)",
        "reacao_curta_1t": "Reação Imediata 1º Turno [−5, +5]",
        "reacao_curta_2t": "Reação Imediata 2º Turno [−5, +5]",
        "entre_turnos": "Período Entre Turnos",
        "reacao_media_1t": "Reação Média [−10, +10]",
        "reacao_ampla_1t": "Reação Ampla [−20, +20]",
        "estendida": "Janela Estendida (Jul–Dez)"
    }

    for jkey, titulo in hm_map.items():
        dj = df_main[df_main["janela"] == jkey]
        if dj.empty:
            continue
            
        pivot = dj.pivot_table(index="setor", columns="ano", values="car", aggfunc="mean")
        pivot_sig = dj.pivot_table(index="setor", columns="ano", values="sig5_simples", aggfunc="any")
        
        annot = pivot.copy().astype(str)
        for r in pivot.index:
            for c in pivot.columns:
                v = pivot.loc[r, c]
                sig = pivot_sig.loc[r, c] if r in pivot_sig.index and c in pivot_sig.columns else False
                annot.loc[r, c] = f"{v:.3f}{'*' if sig else ''}" if pd.notna(v) else ""

        fig, ax = plt.subplots(figsize=(15, 9))
        sns.heatmap(pivot, annot=annot, fmt="", cmap="RdYlGn", center=0, linewidths=0.5, ax=ax)
        ax.set_title(f"CAR por Setor e Ano Eleitoral\n{titulo} (Equal-Weighted) — * p<0.05", 
                     fontsize=14, fontweight="bold", pad=20)
        ax.set_xlabel("Ano Eleitoral")
        ax.set_ylabel("Setor B3")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"heatmap_{jkey}.png"), bbox_inches="tight")
        plt.close()

    log.info("  ✓ Heatmaps gerados")

    # 2. EVOLUÇÃO TEMPORAL
    df_ant = df_main[df_main["janela"] == "antecipacao_45"]
    if not df_ant.empty:
        top_setores = df_ant.groupby("setor")["car"].apply(lambda x: abs(x).mean()).nlargest(5).index.tolist()
        
        fig, axes = plt.subplots(1, 2, figsize=(18, 8))
        for idx, janela in enumerate(["antecipacao_45", "reacao_curta_1t"]):
            ax = axes[idx]
            for setor in top_setores:
                dados = df_main[(df_main["setor"] == setor) & (df_main["janela"] == janela)]
                if not dados.empty:
                    ax.plot(dados["ano"], dados["car"], marker="o", lw=2.5, label=setor)
            ax.axhline(0, color="black", ls="--", alpha=0.4)
            ax.set_title(f"Evolução Temporal - {janela.replace('_', ' ').title()}", fontweight="bold")
            ax.set_xlabel("Ano Eleitoral")
            ax.set_ylabel("CAR (EW)")
            ax.legend(fontsize=9)
            ax.grid(True, alpha=0.3)
        plt.suptitle("Evolução dos CARs nos Top 5 Setores Mais Afetados", fontsize=16, fontweight="bold")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "evolucao_temporal_cars.png"), bbox_inches="tight")
        plt.close()
        log.info("  ✓ Evolução temporal gerada")

    # 3. VOLATILIDADE
    if not df_vol.empty:
        fig, ax = plt.subplots(figsize=(14, 8))
        vol_agg = df_vol.groupby(["setor", "tipo_ano"])["vol_anualizada"].mean().unstack()
        vol_agg.plot(kind="barh", ax=ax, width=0.8, alpha=0.85)
        ax.set_title("Volatilidade Anualizada: Anos Eleitorais vs Não-Eleitorais", fontweight="bold")
        ax.set_xlabel("Volatilidade Anualizada (σ × √252)")
        ax.legend(title="Tipo de Ano")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "volatilidade_comparativa.png"), bbox_inches="tight")
        plt.close()
        log.info("  ✓ Volatilidade comparativa gerada")

    # 4. PLACEBO
    if not df_placebo.empty:
        fig, axes = plt.subplots(1, 2, figsize=(16, 7))
        for idx, (j_real, j_placebo, titulo) in enumerate([
            ("antecipacao_45", "placebo_antecip", "Antecipação"),
            ("reacao_curta_1t", "placebo_reacao", "Reação Curta")
        ]):
            ax = axes[idx]
            real = df_main[df_main["janela"] == j_real]["car"].dropna()
            placebo = df_placebo[df_placebo["janela"] == j_placebo]["car"].dropna()
            
            ax.hist(placebo, bins=40, alpha=0.6, label="Placebo (não-eleitorais)", color="gray")
            ax.hist(real, bins=25, alpha=0.85, label="Eleições Reais", color="steelblue")
            ax.axvline(0, color="red", ls="--", alpha=0.7)
            ax.set_title(f"Distribuição de CARs – {titulo}", fontweight="bold")
            ax.legend()
        plt.suptitle("Teste Placebo: Eleições Reais vs Pseudo-Eventos", fontsize=15, fontweight="bold")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "placebo_test.png"), bbox_inches="tight")
        plt.close()
        log.info("  ✓ Gráfico Placebo gerado")

    # 5. DiD
    if not df_did_agg.empty:
        fig, ax = plt.subplots(figsize=(14, 7))
        for jan in df_did_agg["janela"].unique():
            d = df_did_agg[df_did_agg["janela"] == jan].sort_values("ano")
            ax.plot(d["ano"], d["delta_did"], marker="s", lw=2.5, label=jan.replace("_", " ").title())
            for _, row in d.iterrows():
                if row.get("sig5_did", False):
                    ax.annotate("*", (row["ano"], row["delta_did"]), fontsize=14, color="red", ha="center")
        ax.axhline(0, color="black", ls="--", alpha=0.5)
        ax.set_title("Difference-in-Differences: CAR Estatais − CAR Privadas (* p<0.05)", fontweight="bold")
        ax.set_xlabel("Ano Eleitoral")
        ax.set_ylabel("Δ CAR")
        ax.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "did_estatais_vs_privadas.png"), bbox_inches="tight")
        plt.close()
        log.info("  ✓ Gráfico DiD gerado")

    log.info(f"✓ Todas as visualizações salvas em: {output_dir}")

## MAPA DE RISCO SETORIAL (Gráfico Principal do Estudo)

In [360]:
def gerar_mapa_risco_setorial(df_res, output_dir):
    """
    Mapa de Risco Setorial — Gráfico principal do estudo.
    """
    import matplotlib.pyplot as plt
    import seaborn as sns

    sns.set_style("whitegrid")
    
    # CORREÇÃO AQUI: use "figure.dpi" em vez de "dpi"
    plt.rcParams.update({
        "figure.figsize": (16, 10),
        "font.size": 13,
        "figure.dpi": 300
    })

    janela_principal = "antecipacao_45"
    df_mapa = df_res[(df_res["cenario"] == "completo") & 
                     (df_res["ponderacao"] == "VW") & 
                     (df_res["janela"] == janela_principal)]

    if df_mapa.empty:
        log.warning("Mapa de Risco não gerado (dados insuficientes)")
        return

    pivot = df_mapa.pivot_table(index="setor", columns="ano", values="car", aggfunc="mean")
    pivot_sig = df_mapa.pivot_table(index="setor", columns="ano", values="sig5_simples", aggfunc="any")

    # Ordena do maior risco (mais negativo) para o menor
    setor_risco = pivot.mean(axis=1).sort_values()
    pivot = pivot.loc[setor_risco.index]
    pivot_sig = pivot_sig.loc[setor_risco.index]

    annot = pivot.copy().astype(object)
    for r in pivot.index:
        for c in pivot.columns:
            v = pivot.loc[r, c]
            sig = pivot_sig.loc[r, c] if r in pivot_sig.index and c in pivot_sig.columns else False
            star = "*" if sig else ""
            annot.loc[r, c] = f"{v:+.3f}{star}" if pd.notna(v) else ""

    fig, ax = plt.subplots(figsize=(16, 10))
    sns.heatmap(pivot, annot=annot, fmt="", cmap="RdYlGn", center=0,
                linewidths=0.6, linecolor="white", cbar_kws={"label": "CAR (%)"}, ax=ax)

    ax.set_title("MAPA DE RISCO SETORIAL\n"
                 "CAR na janela de Antecipação (−45 a −1 dias úteis antes do 1º turno)\n"
                 "Equal-Weighted · * = estatisticamente significativo a 5%",
                 fontsize=16, fontweight="bold", pad=25)

    ax.set_xlabel("Ano Eleitoral", fontsize=14, labelpad=15)
    ax.set_ylabel("Setor B3", fontsize=14, labelpad=15)

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "mapa_risco_setorial.png"), bbox_inches="tight")
    plt.close()

    log.info("  ✓ Mapa de Risco Setorial gerado com sucesso!")

## ETAPA 9 – EXPORTAÇÃO E METODOLOGIA

In [361]:
# ============================================================================
# ETAPA 9 – EXPORTAÇÃO
# ============================================================================

def gerar_texto_metodologia(output_dir):
    """Gera arquivo de metodologia pronto para artigo."""
    texto = """# Metodologia - Impacto dos Ciclos Eleitorais na B3 (2002-2022)

## Desenho do Estudo
Estudo de Evento aplicado aos 6 ciclos eleitorais presidenciais brasileiros.

### Principais Características
- Janela de estimação: [-252, -30] dias úteis (MacKinlay, 1997)
- Modelos: CAPM + Fama-French 3 Fatores (NEFIN)
- Ponderação: Equal-Weighted (principal) e Volume-Weighted (robustez)
- Filtros: Liquidez (80%), Penny stocks (R$1,00), Winsorização (1%/99%)
- Testes: t simples, Silva et al. (2015), BMP (1991), HAC-Newey-West
- Robustez: Placebo, DiD (Estatais vs Privadas), Sharpe Ratio

## Limitações
- Viés de sobrevivência (mitigado com mapeamento e filtros)
- Fonte yfinance (lacunas em tickers antigos)
- Não estabelece causalidade, apenas evidência associativa

(Continue com mais detalhes conforme necessário)
"""
    with open(os.path.join(output_dir, "metodologia_limitacoes.md"), "w", encoding="utf-8") as f:
        f.write(texto)
    log.info("  ✓ metodologia_limitacoes.md gerado")


def exportar_resultados(df_res, df_ff3, df_vol, df_sobrev, df_comp, df_diag, 
                        df_placebo, df_did, df_did_agg, df_sharpe, output_dir):
    """Exporta todos os resultados."""
    os.makedirs(output_dir, exist_ok=True)
    
    # CSVs
    df_res.to_csv(os.path.join(output_dir, "resultados_consolidados.csv"), index=False, encoding="utf-8-sig")
    df_sobrev.to_csv(os.path.join(output_dir, "tabela_sobrevivencia.csv"), index=False, encoding="utf-8-sig")
    df_diag.to_csv(os.path.join(output_dir, "diagnostico_download.csv"), index=False, encoding="utf-8-sig")
    
    if not df_ff3.empty:
        df_ff3.to_csv(os.path.join(output_dir, "resultados_ff3.csv"), index=False, encoding="utf-8-sig")
    if not df_placebo.empty:
        df_placebo.to_csv(os.path.join(output_dir, "placebo_results.csv"), index=False, encoding="utf-8-sig")
    if not df_did_agg.empty:
        df_did_agg.to_csv(os.path.join(output_dir, "did_agregado.csv"), index=False, encoding="utf-8-sig")

    # Excel consolidado
    xlsx = os.path.join(output_dir, "analise_ciclos_eleitorais_final.xlsx")
    with pd.ExcelWriter(xlsx, engine="openpyxl") as writer:
        df_res.to_excel(writer, sheet_name="Resultados", index=False)
        df_sobrev.to_excel(writer, sheet_name="Sobrevivência", index=False)
        if not df_ff3.empty:
            df_ff3.to_excel(writer, sheet_name="FF3", index=False)
        if not df_did_agg.empty:
            df_did_agg.to_excel(writer, sheet_name="DiD", index=False)

    log.info(f"  ✓ Arquivos exportados em: {output_dir}")
    gerar_texto_metodologia(output_dir)

In [362]:
# ============================================================================
# ANÁLISE DEDICADA DO SETOR DE PETRÓLEO (ALTAMENTE RECOMENDADA)
# ============================================================================

def analisar_petroleo_dedicado(df_precos, df_volumes, df_empresas, ret_ibov):
    """Roda análise específica só para Petróleo com VW + força total"""
    log.info("=== ANÁLISE DEDICADA DO SETOR DE PETRÓLEO (VW) ===")
    
    # Força todos os tickers relevantes
    tickers_petroleo = []
    for ano_lista in TICKERS_PETROLEO_FORCADOS.values():
        tickers_petroleo.extend(ano_lista)
    tickers_petroleo = list(set(tickers_petroleo))
    
    df_pet = df_precos[tickers_petroleo].copy()
    df_vol_pet = df_volumes[tickers_petroleo].copy()
    
    # Usa a mesma função, mas agora só com o setor de Petróleo
    df_ret_ew_pet, df_ret_vw_pet, _ = construir_indices_setoriais(
        df_pet, df_vol_pet, df_empresas
    )
    
    # Executa event study só com VW
    df_pet_vw = executar_analise(df_ret_vw_pet, ret_ibov, "completo", "VW")
    
    return df_pet_vw

## MAIN – EXECUÇÃO COMPLETA

In [363]:
# ============================================================================
# MAIN - EXECUÇÃO COMPLETA (Versão Final)
# ============================================================================

def main():
    t0 = time.time()
    
    print("\n" + "█"*88)
    print("█  ANÁLISE DE CICLOS ELEITORAIS NA B3 (2002–2022) — VERSÃO FINAL     █")
    print("█"*88 + "\n")

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Etapa 1
    df_emp = carregar_lista_empresas(ARQUIVO_ENTRADA)

    # Etapa 2
    tickers = df_emp["TICKER_YF"].unique().tolist()
    df_precos, df_volumes, df_diag = baixar_precos_yfinance(tickers)
    df_precos = aplicar_filtro_existencia(df_precos, df_emp)

    # Etapa 2c - Índices
    df_ret_ew, df_ret_vw, df_comp = construir_indices_setoriais(df_precos, df_volumes, df_emp)

    # Ibovespa
    ibov = baixar_ibovespa()
    ret_ibov = np.log(ibov / ibov.shift(1)).dropna()

    # Alinhamento
    idx = df_ret_ew.index.intersection(ret_ibov.index)
    df_ret_ew = df_ret_ew.loc[idx]
    df_ret_vw = df_ret_vw.loc[idx]
    ret_ibov = ret_ibov.loc[idx]

    # Tabela de Sobrevivência
    df_sobrev = gerar_tabela_sobrevivencia(df_emp, df_precos)

    # Fatores NEFIN
    df_factors = baixar_fatores_nefin()

    # Event Study CAPM
    df_res_ew = executar_analise(df_ret_ew, ret_ibov, "completo", "EW")
    df_res_vw = executar_analise(df_ret_vw, ret_ibov, "completo", "VW")
    df_resultados = pd.concat([df_res_ew, df_res_vw], ignore_index=True)
    df_resultados = adicionar_benchmarking(df_resultados)

    # Fama-French 3
    df_ff3 = executar_analise_ff3(df_ret_ew, df_factors)

    # Placebo + DiD
    df_placebo = executar_teste_placebo(df_ret_ew, ret_ibov)
    df_did, df_did_agg = executar_did_empresas(df_precos, df_emp, ret_ibov)
        # ... após os outros event studies ...
    df_petroleo = analisar_petroleo_dedicado(df_precos, df_volumes, df_emp, ret_ibov)
    df_petroleo.to_csv(os.path.join(OUTPUT_DIR, "petroleo_dedicado_vw.csv"), index=False)
    # Volatilidade e Sharpe
    df_vol = calcular_volatilidade_comparativa(df_ret_ew)
    df_sharpe = calcular_sharpe_comparativo(df_ret_ew)

    # Visualizações
    gerar_visualizacoes(df_resultados, df_vol, df_placebo, df_did_agg, df_sharpe, OUTPUT_DIR)

    # Mapa de Risco Setorial (o mais importante)
    gerar_mapa_risco_setorial(df_resultados, OUTPUT_DIR)

    # Exportação
    exportar_resultados(df_resultados, df_ff3, df_vol, df_sobrev, df_comp,
                        df_diag, df_placebo, df_did, df_did_agg, df_sharpe, OUTPUT_DIR)

    # Resumo final
    elapsed = time.time() - t0
    print("\n" + "="*88)
    print("ANÁLISE CONCLUÍDA COM SUCESSO!")
    print("="*88)
    print(f"Tempo total: {elapsed/60:.1f} minutos")
    print(f"Arquivos salvos em: {OUTPUT_DIR}")
    print(f"→ Mapa de Risco Setorial gerado: mapa_risco_setorial.png")
    print("="*88)


if __name__ == "__main__":
    main()

16:25:09 [INFO] ======================================================================
16:25:09 [INFO] ETAPA 1: INGESTÃO DE DADOS
16:25:09 [INFO] ======================================================================
16:25:09 [INFO]   → 696 empresas | 11 setores | 2 remapeados | 31 estatais
16:25:09 [INFO]     [ESTATAL] TELB4 — TELEC. BRASILEIRAS S.A. - TELEBRÁS (Comunicações)
16:25:09 [INFO]     [ESTATAL] BAGE3 — BANRISUL ARMAZENS GERAIS SA (Construção e Transporte)
16:25:09 [INFO]     [ESTATAL] BAZA3 — BANCO DA AMAZÔNIA S.A. (Financeiro)
16:25:09 [INFO]     [ESTATAL] BBAS3 — BANCO DO BRASIL S.A. (Financeiro)
16:25:09 [INFO]     [ESTATAL] BGIP4 — BANCO DO ESTADO DE SERGIPE SA (Financeiro)
16:25:09 [INFO]     [ESTATAL] BPAR3 — BANCO DO ESTADO DO PARÁ S/A. (Financeiro)
16:25:09 [INFO]     [ESTATAL] BRSR6 — BANCO DO ESTADO DO RIO GRANDE DO SUL SA (Financeiro)
16:25:09 [INFO]     [ESTATAL] BNBR3 — BANCO DO NORDESTE DO BRASIL SA (Financeiro)
16:25:09 [INFO]     [ESTATAL] BEES4 — BANESTES S


████████████████████████████████████████████████████████████████████████████████████████
█  ANÁLISE DE CICLOS ELEITORAIS NA B3 (2002–2022) — VERSÃO FINAL     █
████████████████████████████████████████████████████████████████████████████████████████



16:25:11 [ERROR] 
20 Failed downloads:
16:25:11 [ERROR] ['KRSA3.SA', 'ALRS3.SA', 'BRQS3.SA', 'ELEA3.SA', 'NWTL3.SA', 'ELTR3.SA', 'FORP3.SA', 'CPTP3.SA', 'BION3.SA', 'PTCA3.SA', 'PRCM3.SA', 'SULB3.SA', 'IGBR3.SA', 'RBNS11.SA', 'CLAR3.SA', 'CLSA3.SA', 'UNDA3.SA', 'ALGT3.SA']: YFTzMissingError('possibly delisted; no timezone found')
16:25:11 [ERROR] ['EMBJ3.SA', 'CTAX3.SA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2001-01-01 -> 2023-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 978314400, endDate = 1703991600")')
16:25:11 [INFO]   Bloco 2/13 (50 tickers)
16:25:13 [ERROR] 
26 Failed downloads:
16:25:13 [ERROR] ['AFLU3.SA', 'RUMO3.SA', 'ARTR3.SA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2001-01-01 -> 2023-12-31)')
16:25:13 [ERROR] ['BAGE3.SA', 'TGMC3.SA', 'RMNO3.SA', 'MEND5.SA', 'MRSA3.SA', 'MOTI3.SA', 'VTAL3.SA', 'STBP3.SA', 'SULP4.SA', 'VERO3.SA', 'IVPR3.SA', 'PPFX3.SA', 'RMOE3.SA', 'AZEV11.SA', 'SQIA3.SA', 'AFDI3.SA', 'SNR

NameError: name 'SETORES_ESPECIAIS' is not defined